### 提升（boosting）
给定输入向量x和输出向量y组成的若干训练样本(x1, y1)，(x2, y2)...(xn, yn)，目标是找到近似函数F^(x)，使得损失函数L(y, F(x))的损失函数最小。

$L(y, F(x))$的典型定义：
    
$L(y, F(x)) = \frac{1}{2}(y-F(x))^2$ （假设y服从高斯分布）

$L(y, F(x)) = |y-F(x)|$ （假设y服从拉普拉斯分布）

假定最优函数为F* (x)，即：

$F^*(x) = argminE_{(x, y)}[L(y, F(x))]$

假定F(x)是一族基函数$f_{i}(X)$的加权和

$F(x) = sum_{i=1}^M r_{i}f_{i}(x) + const $

#### 提升算法推导
梯度提升方法寻找最优解F(x)，使得损失函数在训练集上的期望最小。

首先，给定常函数$F_{0}(x)$:
$F_{0}(x) = argmin_{r} \sum_{i=1}^n L(y_{i}, r)$

以贪心的思路扩展得到$F_{m}(x)$

$F_{m}(x) = F_{m-1}(x) + argmin_{f \in{H}} \sum_{i=1}^n L(y_{i}, F_{m-1}(x) + f(x))$

这里给出一个形象化的例子：

第一颗树给出预测值，第二棵树会对残差部分进行估计，估计残差再和残差继续做差，第三棵树同样对残差进行估计...

### GBDT（梯度提升决策树）

梯度提升的典型基函数即决策树（尤其是CART）

第m步的梯度提升是根据残差数据计算决策树$t_{m}(x)$。令树$t_{m}(x)$的叶节点数据为J，即树$t_{m}(x)$将输入空间划分为J，即树$t_{m}(x)$将输入空间划分为J个不相交区域$R_{1m}, R_{2m},...,R_{Jm}$，并且决策树$t_{m}(x)$可以在每个区域中给出某个类型的确定性预测。使用指示记号I(x)，对于输入x，$t_{m}(x)$为：
<img src="./imgs/img06.png" width = "200" height = "200" />

使用线性搜索计算学习率，最小化损失函数

$F_{m}(x)=F_{m-1}(x) + \gamma \cdot t_{m}(x_{i})$

$r_{m}=argmin_{r}\sum_{i=1}^{n}L(y_{i}, F_{m-1}(x_{i}) + \gamma \cdot t_{m}(x_{i}))$

进一步，对树的每个区域分别计算步长，从而系数$b_{jm}$被合并到步长中，从而：

$F_{m}(x) = F_{m-1}(x)+\sum_{j=1}^{J}\gamma{jm}I(x\in R_{jm})$

$\gamma_{jm}=argmin_{r}\sum_{x_{i}\in R_{jm}}L(y_{i}, F_{m-1}(x_{i}) + \gamma \cdot t_{m}(x_{i}))$

（这里要强调一个容易混淆的概念，L是残差的损失函数，$F_{m-1}(x_{i})$是上一棵树的预测值$argmin_{r}\sum_{x_{i}\in R_{jm}}L(y_{i}, F_{m-1}(x_{i}) + \gamma \cdot t_{m}(x_{i}))$是目标函数）

GBDT并非对整个树做权值的调整，而是对每一个叶子做权值的调整。

#### 参数设置和正则化

- 1.对训练集拟合过高会降低模型的泛化能力，需要使用正则化来降低过拟合
    - 1.1 对复杂模型增加惩罚项，如：模型复杂度正比于叶节点数目或叶节点预测值的平方和
    - 1.2 对决策树剪枝
- 2.叶节点数目控制了树的层数，一般选择4<=J<=8
- 3. 叶节点包含的最小样本数目
    - 3.1 防止出现过小的叶节点，降低预测的方差
- 4.梯度提升迭代次数M
    - 4.1 增加M可降低训练集的损失值，但有过拟合风险
    - 4.2 交叉验证
    
#### GBDT总结
- 1.损失函数时最小平方误差、绝对值误差等，则为回归问题；而损失函数为交叉熵或KL-散度，则为分类问题。

- 2.对目标函数分解成若干基函数的加权和，是常见的技术手段：神经网络、径向基函数、傅里叶小波变换、SVM等都可以看到它的影子。

- 3.只使用一阶导数信息。

- 4.GBDT每一轮关注真实值和预测值的残差，然后下一轮以本轮的残差为输入，尽量去拟合这个残差，使得每一轮的残差不断减小。所以GBDT一定向损失函数减小的方向变化，而传统的boosting算法只能尽可能的向梯度方向减小。这是GBDT相对于传统boosting的算法的最大区别，这也是为什么GBDT相比传统Boosting算法可以用更少的树个数与深度达到更好的效果。

### Xgboost核心推导过程

#### 考虑使用二阶导信息
求第t颗决策树的目标函数，$J(f)$是Xgboost的损失函数。

$f_{t}(x_{i})$就是第t棵树对样本$x_{i}$的预测值，预测值本身是残差部分。

<img src="./imgs/img01.png" width = "350" height = "350" />

其中目标函数的解释，
<img src="./imgs/img02.jpg" width = "350" height = "350" />

最开始一棵树，然后如果是MAE选用中位数预测y，如果是MSE选用平均数预测y，然后第二棵树就在这个基础上再次求导。

#### 正则项的定义
决策树的复杂度可考虑叶结点数和叶权值，如使用叶结点总数和叶权值平方和的加权。
这里$w_{j}$为叶子节点的预测值
$\Omega(f_{t}) = \Upsilon T + \frac{1}{2} \lambda \sum_{j=1}^T w_{j}^{2}$

#### 目标函数计算
<img src="./imgs/img03.png" width = "300" height = "300" />

n--------样本个数

$y_{i}$--------第i个样本的真实值

$y_{i}^{^{t-1}}$--------第t-1个决策树的第i个样本的预测值

T--------当前叶子节点个数

$\sum_{i\in I_{j}}g_{i}$--------位于j号叶子的所有样本的梯度的和

$f_{t}(x_{i})$--------第i个样本的预测值

$w_{q}(x_{i})$--------第i个样本在哪个叶子（与上面的等价）

#### 目标函数继续化简
<img src="./imgs/img04.png" width = "300" height = "300" />

#### 构造树的结构
问：对于当前节点，如何进行子树划分？
<img src="./imgs/img05.png" width = "300" height = "300" />

借鉴ID3/C4.5/CART的做法，使用贪心法：
    
    1）对于某可行划分，计算划分后的J(f)
    2)对于所有可行划分，选择J(f)降低到最小的分割点

以上就是XGBoost的核心推导过程。
相对于传统的GBDT，XGBoost使用了二阶信息，可以更快地在训练集上收敛。

#### 总结
xgboost采用第一棵树取均值，第二颗树真实值和第二颗树的预测值的差值重新构建树，树的代价函数根据Taylor展式进行转换，新的代价函数包含一阶导和二阶导信息。令损失函数相对于$w_{j}$的导数为0可以求出最小化损失函数各个叶子节点的预测值。通过将预测值带入损失函数可求得损失函数的最小值，容易计算节点分裂前后损失函数的差值。Xgboost就是采用最大化这个差值为准则来进行决策树的构建。

#### Xgboost与GBDT的区别

1. GBDT算法基于成本函数的负梯度来构造新的决策树，只有在决策树完成构建后才进行剪枝，而XGBoost在决策树构建阶段就加入了正则项。

2. GBDT在模型训练时只使用了代价函数的一阶导信息，而XGBoost对代价函数进行二阶泰勒展开，可以同时使用一阶和二阶导数。

3. 传统的GBDT使用CART作为基分类器，而XGBoost支持多种类型的基分类器，比如线性分类器。

4. 传统的GBDT在每轮迭代使用全部数据，而XGBoost使用随机森林相似的策略，支持对数据的采样。

5. 传统的GBDT没有设计对缺失值的处理，XGBoost能够学习出缺失值处理策略。

### 使用随机森林计算特征重要度
随机森林是常用的衡量特征重要性的方法。
计算正例经过的节点，使用经过节点的样本数目、经过节点的gini系数和等指标。或者随机替换一列数据，重新建立决策树，计算模型的正确率的变换，从而考虑这一列的特征重要度。
特征重要度的判断依据可以是：

    1）经过节点的样本数目
    2）经过节点的gini系数和
    3）随机替换一列特征，重新构树，计算模型正确率变化

### 孤立森林（Isolation Forest）
从根节点要叶子节点的路径长度，可以度量样本是不是噪声的一个指标。
随机选择特征、随机选择分割点，生成一定深度的决策树，若干颗iTree组成iForest
    
    1）计算iTree中样本x从跟到叶子节点的长度f(x)
    2）计算iForest中f(x)的总和F(x)
若样本x是异常值，它应该在大多数的iTree中很快到达叶子，即F(x)较小。